In [2]:
import pandas as pd
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pickle


In [3]:

# Load data
data_map = pd.read_csv('C:/Users/DELL/OneDrive/Documents/GitHub/xray/data/data_map.csv')

# Label encoding
lbl_encoder = LabelEncoder()
data_map['clas_enc'] = lbl_encoder.fit_transform(data_map['clas'])


In [4]:

class XrayDataset(Dataset):
    def __init__(self, df_split, transform):
        self.df_split = df_split
        self.transform = transform

    def __len__(self):
        return len(self.df_split)

    def __getitem__(self, index):
        img_path = self.df_split.iloc[index]['resized']
        clas = self.df_split.iloc[index]['clas_enc']
        img = Image.open(img_path).convert('L')
        img_trans = self.transform(img)
        return img_trans, clas



In [5]:


train_dataset = XrayDataset(data_map[data_map['split']=='train'], transform=transforms.ToTensor())
val_dataset = XrayDataset(data_map[data_map['split']=='val'], transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

loss = nn.CrossEntropyLoss()


In [10]:
def train_model(model, name):

    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(6):
        model.train()
        runinng_loss = 0
        for imgs, lbls in train_loader:
            lbls = lbls.long()
            optimizer.zero_grad()
            preds = model(imgs)
            loss_val = loss(preds, lbls)
            loss_val.backward()
            optimizer.step()
            runinng_loss += loss_val.item()
        print(runinng_loss / len(train_loader))

    torch.save(model.state_dict(), f'C:/Users/DELL/OneDrive/Documents/GitHub/xray/models/{name}.pth')





In [9]:
res18 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
res18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
res18.fc = nn.Linear(512, 2)

train_model(res18, "model_resnet18")

0.1797138425504922
0.08694728737464175
0.06915919633285653
0.04235415577429757
0.03803665490548675
0.03750917100058446
0.03256917081056426
0.039328805896946595
0.01468180744816328
0.02734086243162892


In [11]:
res34 = models.resnet34(weights=models.ResNet34_Weights.IMAGENET1K_V1)
res34.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
res34.fc = nn.Linear(512, 2)

train_model(res34, "model_resnet34")

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\resnet34-b627a593.pth


100.0%


0.22452396309624115
0.114255377834308
0.08375857937512808
0.06467081385978547
0.056087172841043434
0.045056366894035214


In [12]:
mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
mobilenet.classifier[1] = nn.Linear(1280, 2)

train_model(mobilenet, "model_mobilenet_v2")

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to C:\Users\DELL/.cache\torch\hub\checkpoints\mobilenet_v2-b0353104.pth


100.0%


0.21572489752101578
0.09659420128931136
0.08929489873948374
0.0580972064979592
0.055852364259841296
0.0801637761844523


In [13]:

with open(r'C:\Users\DELL\OneDrive\Documents\GitHub\xray\data\lbl_encoder.pkl', 'wb') as f:
    pickle.dump(lbl_encoder, f)
